In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
from pprint import pprint
import time
from scipy.stats import linregress
import reverse_geocoder as rg

In [2]:
# Set base url parameters
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
magnitude = "minmagnitude=5"
limit = "limit=19999"
year=1990

# set up lists to hold reponse info
time_ls = []
lat_ls = []
lng_ls = []
depth_ls = []
mag_ls = []
sig_ls = []
place_ls = []

# iterate through years
for x in range (0,10):
    time = f"starttime={year}-01-01&endtime={year}-12-31"
    query_url = (f"{base_url}&{time}&{magnitude}&{limit}")
    response = requests.get(query_url).json()

    # Append data to appropriate list
    for i in range (0, len(response["features"])):
        mag_ls.append(response["features"][i]["properties"]["mag"])
        place_ls.append(response["features"][i]["properties"]["place"])
        time_ls.append(response["features"][i]["properties"]["time"])
        sig_ls.append(response["features"][i]["properties"]["sig"])
        lat_ls.append(response["features"][i]["geometry"]["coordinates"][1])
        lng_ls.append(response["features"][i]["geometry"]["coordinates"][0])
        depth_ls.append(response["features"][i]["geometry"]["coordinates"][2])
    # counter for next year
    
    year=year+1

In [5]:
# create dictionary
data = {
    "Time": time_ls,
    "Lat": lat_ls,
    "Lng": lng_ls,
    "Depth": depth_ls,
    "Mag": mag_ls,
    "Sig": sig_ls,
    "Place": place_ls
}
# create dataframe
data_df = pd.DataFrame(data)

#convert timestamp to dates
data_df["Time"] = pd.to_datetime(data_df["Time"], unit="ms")
data_df["Month"] = pd.DatetimeIndex(data_df["Time"]).month
data_df["Day"] = pd.DatetimeIndex(data_df["Time"]).day
data_df["Year"] = pd.DatetimeIndex(data_df["Time"]).year

# Print dataframe

data_df= data_df[["Lat","Lng","Depth","Mag","Sig","Place","Time","Month","Day","Year"]]

data_df.head()

,Lat,Lng,Depth,Mag,Sig,Place,Time,Month,Day,Year
0,18.668,-107.016,33.0,5.2,416,"off the coast of Jalisco, Mexico",1990-12-30 21:30:29.200,12,30,1990
1,-5.097,150.967,178.6,7.5,865,"New Britain region, Papua New Guinea",1990-12-30 19:14:18.930,12,30,1990
2,-20.737,-173.493,33.0,5.1,400,Tonga,1990-12-30 14:31:43.860,12,30,1990
3,11.265,139.584,65.2,5.1,400,"State of Yap, Federated States of Micronesia",1990-12-30 11:47:40.950,12,30,1990
4,23.944,121.860,10.0,5.2,416,Taiwan,1990-12-30 10:31:17.570,12,30,1990


In [6]:
data_df.to_csv("output_data/earthquake_data_1990-99.csv", index= False)